In [ ]:
from pathlib import Path

import uproot
import awkward as ak
from coffea import nanoevents

import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh.processors.utils import GEN_FLAGS, PDGID, pad_val
from bbtautau.processors.GenSelection import gen_selection_HHbbtautau
from bbtautau import bbtautau_vars

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
# MAIN_DIR = Path("../../")

# plot_dir = MAIN_DIR / "plots/TriggerStudy/24Nov6"
# plot_dir.mkdir(parents=True, exist_ok=True)

year = "2022"

In [ ]:
events = nanoevents.NanoEventsFactory.from_root(
    "root://cmseos.fnal.gov//store/user/lpcdihiggsboost/NanoAOD_v12_ParT/rkansal/2022/HHbbtt/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV/241028_235514/0000/Run3Summer22NanoAODv12_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

In [ ]:
gen_vars = gen_selection_HHbbtautau(events, events.FatJet, None)

In [ ]:
trig_dR = 0.2

In [ ]:
def trig_match_sel(leptons, trig_leptons, year, trigger, filterbit, ptcut, trig_dR=0.2):
    """
    Returns selection for leptons which are trigger matched to the specified trigger.
    """
    trigger = bbtautau_vars.HLT_dict[year][trigger][0][4:]
    trig_fired = heevents.HLT[trigger]
    print(f"{trigger} rate: {ak.mean(trig_fired)}")

    filterbit = 2**filterbit

    pass_trig = (trig_leptons.filterBits & filterbit) == filterbit
    trig_l = trig_leptons[pass_trig]
    trig_l_matched = ak.any(leptons.metric_table(trig_l) < trig_dR, axis=2)
    trig_l_sel = trig_fired & trig_l_matched & (leptons.pt > ptcut)
    return trig_l_sel

## Testing Electron Selection

In [ ]:
heevents = events[gen_vars["GenTauhe"]]
len(heevents) / len(events)

In [ ]:
leptons = heevents.Electron
trigobj = heevents.TrigObj
lsel = (
    leptons.mvaIso_WP90
    & (leptons.pt > 20)
    & (abs(leptons.eta) < 2.5)
    & (abs(leptons.dz) < 0.2)
    & (abs(leptons.dxy) < 0.045)
)
leptons = leptons[lsel]

In [ ]:
# Trigger: (filterbit, ptcut for matched lepton)
triggers = {"EGamma": (1, 31), "ETau": (6, 25)}
trig_leptons = trigobj[trigobj.id == PDGID.e]

TrigMatchDict = {
    f"ElectronTrigMatch{trigger}": trig_match_sel(
        leptons, trig_leptons, year, trigger, filterbit, ptcut
    )
    for trigger, (filterbit, ptcut) in triggers.items()
}

In [ ]:
lw = 2
bins = np.arange(0, 4)
plt.hist(ak.count(leptons.pt, axis=1), bins=bins, histtype="step", label="Electrons", linewidth=lw)
plt.hist(
    ak.count(leptons.pt[TrigMatchDict["ElectronTrigMatchEGamma"]], axis=1),
    bins=bins,
    histtype="step",
    label="EGamma",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons.pt[TrigMatchDict["ElectronTrigMatchETau"]], axis=1),
    bins=bins,
    histtype="step",
    label="ETau",
    linewidth=lw,
)
plt.legend()
plt.show()

## Testing Muon Selection

In [ ]:
heevents = events[gen_vars["GenTauhm"]]
len(heevents) / len(events)

In [ ]:
leptons = heevents.Muon
trigobj = heevents.TrigObj

In [ ]:
lsel = (
    leptons.tightId
    & (leptons.pt > 20)
    & (abs(leptons.eta) < 2.4)
    & (abs(leptons.dz) < 0.2)
    & (abs(leptons.dxy) < 0.045)
)
leptons = leptons[lsel]

In [ ]:
# Trigger: (filterbit, ptcut for matched lepton)
triggers = {"Muon": (3, 26), "MuonTau": (6, 22)}
trig_leptons = trigobj[trigobj.id == PDGID.mu]

TrigMatchDict = {
    f"MuonTrigMatch{trigger}": trig_match_sel(
        leptons, trig_leptons, year, trigger, filterbit, ptcut
    )
    for trigger, (filterbit, ptcut) in triggers.items()
}

In [ ]:
lw = 2
bins = np.arange(0, 4)

plt.figure(figsize=(6, 6))
plt.hist(ak.count(leptons.pt, axis=1), bins=bins, histtype="step", label="Muons", linewidth=lw)
plt.hist(
    ak.count(leptons.pt[TrigMatchDict["MuonTrigMatchMuon"]], axis=1),
    bins=bins,
    histtype="step",
    label="Muon",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons.pt[TrigMatchDict["MuonTrigMatchMuonTau"]], axis=1),
    bins=bins,
    histtype="step",
    label="MuonTau",
    linewidth=lw,
)
plt.legend()
plt.show()

## Testing Tau Selection

In [ ]:
heevents = events[gen_vars["GenTauhe"]]
len(heevents) / len(events)

In [ ]:
leptons = heevents.Tau
trigobj = heevents.TrigObj

In [ ]:
lsel = (
    (leptons.idDeepTau2018v2p5VSjet >= 5)
    # & (leptons.idDeepTau2018v2p5VSe >= 3)
    & (leptons.pt > 20)
    & (abs(leptons.eta) < 2.5)
    & (abs(leptons.dz) < 0.2)
)
leptons = leptons[lsel]

In [ ]:
trig_leptons = trigobj[trigobj.id == PDGID.tau]

# Trigger: (filterbit, ptcut for matched lepton)
triggers = {"SingleTau": (10, 185), "DiTau": (7, 37), "ETau": (8, 32), "MuonTau": (9, 30)}

TrigMatchDict = {
    f"TauTrigMatch{trigger}": trig_match_sel(leptons, trig_leptons, trigger, filterbit, ptcut)
    for trigger, (filterbit, ptcut) in triggers.items()
}

In [ ]:
TrigMatchDict

In [ ]:
leptons

In [ ]:
lw = 2

plt.figure(figsize=(10, 6))
plt.hist(
    ak.count(leptons.pt, axis=1),
    bins=np.arange(0, 4),
    histtype="step",
    label="Taus",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons[trig_1tau_sel].pt, axis=1),
    bins=np.arange(0, 4),
    histtype="step",
    label="1tau",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons[trig_ditau_sel].pt, axis=1),
    bins=np.arange(0, 4),
    histtype="step",
    label="ditau",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons[trig_etau_sel].pt, axis=1),
    bins=np.arange(0, 4),
    histtype="step",
    label="etau",
    linewidth=lw,
)
plt.hist(
    ak.count(leptons[trig_mtau_sel].pt, axis=1),
    bins=np.arange(0, 4),
    histtype="step",
    label="mtau",
    linewidth=lw,
)

plt.legend()
plt.yscale("log")
plt.show()

Testing Jet Selections

In [110]:
jets = events.Jet

In [122]:
jetidtightbit = (jets.jetId & 2) == 2
jetidtight = (
    ((np.abs(jets.eta) <= 2.7) & jetidtightbit)
    | (
        ((np.abs(jets.eta) > 2.7) & (np.abs(jets.eta) <= 3.0))
        & jetidtightbit
        & (jets.neHEF >= 0.99)
    )
    | (((np.abs(jets.eta) > 3.0)) & jetidtightbit & (jets.neEmEF < 0.4))
)

jetidtightlepveto = (
    (np.abs(jets.eta) <= 2.7) & jetidtight & (jets.muEF < 0.8) & (jets.chEmEF < 0.8)
) | ((np.abs(jets.eta) > 2.7) & jetidtight)

In [124]:
ak.mean(jets.isTight == jetidtight)

0.9525101946215394

In [121]:
jetidtight

<Array [[True, False, True, ... True, False]] type='25300 * var * bool'>